# 환경구축

데이터 파일들을 불러오고 저장하기 위해서 Google Drive를 연결합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.condarc
./root/.bin.priority/
./root/.bin.priority/pip2
./root/.bin.priority/pip
./root/.bin.priority/pip3
./root/.tmux.conf
./root/.profile
./root/.vimrc
./root/.bashrc.biolab
--2022-06-09 14:04:48--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M  99.7MB/s    in 0.7s    

2022-06-09 14:04:49 (99.7 MB/s) - ‘miniconda3.sh’ sav

In [3]:
!md5sum drive/MyDrive/binfo1-datapack1/*

140aaf30bcb9276cc716f8699f04ddd6  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam.bai
9f43477870337dfddbc154809a2a3ec8  drive/MyDrive/binfo1-datapack1/CLIP-let7d.bam
8f0013aa795510f597e241c1b69e6341  drive/MyDrive/binfo1-datapack1/CLIP-let7d-gene.pileup
8f0013aa795510f597e241c1b69e6341  drive/MyDrive/binfo1-datapack1/CLIP-let7d.pileup
d0bc27a4926b599fb905ce2c530f259b  drive/MyDrive/binfo1-datapack1/CLIP-let7d_sorted.bam
d68c64201fb0c3c301fbb1efdde8311d  drive/MyDrive/binfo1-datapack1/CLIP-let7d_sorted.bam.bai
d5d8996d383d994fe1916bf239f91964  drive/MyDrive/binfo1-datapack1/CLIP-let7f-1.bam
1c34f27bcc5f407e54d8112938bc9d2c  drive/MyDrive/binfo1-datapack1/CLIP-let7f-1-gene.pileup
9b172b22ce452b8f7ef1b02dcf61516b  drive/MyDrive/binfo1-datapack1/CLIP-let7f-1.pileup
0d33bf4b3a5ce22ba13a62323efd6883  drive/MyDrive/binfo1-datapack1/CLIP-let7g.bam
4ea14471c5d4bf9ab1fd91b9a5b2193c  drive/MyDrive/binfo1-datapack1/CLI

실습에 필요한 프로그램들 몇 가지를 추가로 설치합니다.

In [4]:
!conda install -y bedtools bioawk

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bioawk-1.0                 |       h7132678_7         196 KB  bioconda
    ------------------------------------------------------------
                                           Total:         196 KB

The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7



bioawk-1.0           | 196 KB    | : 100% 1.0/1 [00:00<00:00,  3.96it/s]                
Prepar

# Fig S2A 재현해보기
error가 많이 나오는 부분의 unique sequence들의 각각 개수를 세서 genome browser로 표현하고, 점수 트랙을 같이 표시

In [5]:
%cd /content/drive/MyDrive/binfo1-datapack1/

/content/drive/MyDrive/binfo1-datapack1


논문은 예전 레퍼런스 유전체인 mm9을 쓰고 있기 때문에, 현재 사용되는 레퍼런스 mm39와는 좌표계에 차이가 있습니다. 새로 annotation에서 위치를 찾습니다.

In [6]:
!grep -i mirlet7d gencode.gtf

chr13	ENSEMBL	gene	48689488	48689590	.	-	.	gene_id "ENSMUSG00000065453.3"; gene_type "miRNA"; gene_name "Mirlet7d"; level 3; mgi_id "MGI:2676796";
chr13	ENSEMBL	transcript	48689488	48689590	.	-	.	gene_id "ENSMUSG00000065453.3"; transcript_id "ENSMUST00000083519.3"; gene_type "miRNA"; gene_name "Mirlet7d"; transcript_type "miRNA"; transcript_name "Mirlet7d-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676796"; tag "basic";
chr13	ENSEMBL	exon	48689488	48689590	.	-	.	gene_id "ENSMUSG00000065453.3"; transcript_id "ENSMUST00000083519.3"; gene_type "miRNA"; gene_name "Mirlet7d"; transcript_type "miRNA"; transcript_name "Mirlet7d-201"; exon_number 1; exon_id "ENSMUSE00000522678.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676796"; tag "basic";


Splicing 되지 않는 transcript이기 때문에 결과가 단조롭습니다. 해당 좌표 범위로 bam을 솎아냅니다.

In [7]:
!samtools view -b -o CLIP-let7d.bam CLIP-35L33G.bam chr13:48689488-48689590
!samtools view CLIP-let7d.bam | wc -l

187


samtools를 이용해 bam file을 sort하고 index file (bam.bai) 생성

In [8]:
!samtools sort CLIP-let7d.bam -o CLIP-let7d_sorted.bam

In [9]:
!samtools index CLIP-let7d_sorted.bam

각 자리에 mapping된 read들의 분포를 요약하기 위해 samtools mpileup 이용하기.

In [10]:
!samtools mpileup CLIP-let7d.bam > CLIP-let7d.pileup
!wc -l CLIP-let7d.pileup

[mpileup] 1 samples in 1 input files
87 CLIP-let7d.pileup


Mirlet7d의 핵심부분을 살펴봐서 어떻게 하면 좋을지 견적을 내보기.

In [11]:
!head CLIP-let7d.pileup

chr13	48689488	N	24	^Ga^Ia^Ha^Ia^Ha^Ia^Ia^Ia^Ga^Ia^Ia^Ia^Ia^Ia^Ha^Ia^Ga^Ga^Ga^Ia^Ea^Da^Ha^Ga	BIHGGIHIEIIIGIHIGEGH5BDD
chr13	48689489	N	53	aaaaaaaaaaaaaaaaaaaaaaaaa^Ia^Ha^Ia^Ga^Ga^Ha^Ia^Ia^Ga^Ga^Ia^Ia^Ha^Ia^Ia^Ha^Ha^Da^Ia^Ha^Ia^Ia^Ia^Ia^Ia^Ha^Ga^Ga	GIHIGIIIEIIIIIHIG>EB4@5HDIHIBEGIID?IIGII@H>IGHIIIGHDC
chr13	48689490	N	69	ttttttttttttttttttttttttttttttttttttttttttttttttttttt^Gt^Gt^Ht^Gt^It^It^Ht^It^Gt^Gt^Ht^It^It^It^Ht^It	GIHIGHIFFIIIIIFHGEDG;2>FDIHIDGBIIDBIIDIIAH>IBGIIIGHBG=BG=IIGIDGHFIIHI
chr13	48689491	N	105	aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa^Ia^Ia^Ha^Ga^Ia^Ga^Ia^Ga^Ha^Ha^Ha^Ga^Ia^Ha^Ha^Ia^@a^Ia^Ia^Ia^Ga^Ha^Ha^Ga^Ha^Ga^Ia^Ha^Ga^Ia^Ga^Ha^Ga^Ia^Ia^Ga	EIGHEHHIEIIIIIHIGCBH:?>HGGHI>GBIIGBIIGIHBB;IHIIIIGHDC3=G:IGGIDEHIFIGHII=GIDHEGHHEIGGI7III9=HGBAIGBIGFG>I2
chr13	48689492	N	105	aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa	GIGIEIEI7IIIGIGIG;?F:D@G7GHIAGAIIG3IIGIHDB2IHIIIIDDGC1<GCIIHIGCHIIIEIIIGCI

In [12]:
!awk '$2 >= 48689488 && $2 <= 48689590 { print $0; }' CLIP-let7d.pileup > CLIP-let7d-gene.pileup
!tail CLIP-let7d-gene.pileup

chr13	48689565	N	6	cccccc	>IGIIH
chr13	48689566	N	6	tttttt	:IGFIH
chr13	48689567	N	6	aaaaaa	8IFIFI
chr13	48689568	N	6	cccccc	?IGIII
chr13	48689569	N	6	tttttt	;IGIII
chr13	48689570	N	5	aaaaa	EDECD
chr13	48689571	N	6	cccccc	:GGIGG
chr13	48689572	N	6	cccccc	:GGIGG
chr13	48689573	N	6	tttttt	5GGIGG
chr13	48689574	N	6	c$c$c$c$c$c$	;GGIGG


In [13]:
!ls

CLIP-35L33G.bam			       gencode-exons-containing-startcodon.bed
CLIP-35L33G.bam.bai		       gencode.gtf
CLIP-let7d.bam			       gencode-plusexon.gtf
CLIP-let7d-gene.pileup		       gencode-start.gtf
CLIP-let7d.pileup		       Mirlet7d.txt
CLIP-let7d_sorted.bam		       Mirlet7f-1.txt
CLIP-let7d_sorted.bam.bai	       Mirlet7g.txt
CLIP-let7f-1.bam		       read-counts.txt
CLIP-let7f-1-gene.pileup	       read-counts.txt.summary
CLIP-let7f-1.pileup		       RNA-control.bam
CLIP-let7g.bam			       RNA-control.bam.bai
CLIP-let7g-gene.pileup		       RNA-siLin28a.bam
CLIP-let7g.pileup		       RNA-siLin28a.bam.bai
filtered-RPF-siLin28a.bam	       RNA-siLuc.bam
filtered-RPF-siLuc.bam		       RNA-siLuc.bam.bai
fivepcounts-filtered-RPF-siLin28a.txt  RPF-siLin28a.bam
fivepcounts-filtered-RPF-siLuc.txt     RPF-siLin28a.bam.bai
fivepcounts-RPF-siLin28a.bed	       RPF-siLuc.bam
fivepcounts-RPF-siLuc.bed	       RPF-siLuc.bam.bai


이제 pandas에 불러와서 분석하기.

In [14]:
import pandas as pd
dpileup = pd.read_csv('CLIP-let7d-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
dpileup.tail()

,chrom,pos,_ref,count,basereads,quals
82,chr13,48689570,N,5,aaaaa,EDECD
83,chr13,48689571,N,6,cccccc,:GGIGG
84,chr13,48689572,N,6,cccccc,:GGIGG
85,chr13,48689573,N,6,tttttt,5GGIGG
86,chr13,48689574,N,6,c$c$c$c$c$c$,;GGIGG


여기서 pileup에서 계산에 사용할 것은 match와 substitution이기 때문에 나머지에 해당하는 tag들 제거하기.

In [15]:
import re
toremove = re.compile('[<>$*#^]')
dpileup['matches'] = dpileup['basereads'].apply(lambda x: toremove.sub('', x))

In [16]:
dpileup[['chrom', 'pos', 'matches']]

,chrom,pos,matches
0,chr13,48689488,GaIaHaIaHaIaIaIaGaIaIaIaIaIaHaIaGaGaGaIaEaDaHaGa
1,chr13,48689489,aaaaaaaaaaaaaaaaaaaaaaaaaIaHaIaGaGaHaIaIaGaGaI...
2,chr13,48689490,tttttttttttttttttttttttttttttttttttttttttttttt...
3,chr13,48689491,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
4,chr13,48689492,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
...,...,...,...
82,chr13,48689570,aaaaa
83,chr13,48689571,cccccc
84,chr13,48689572,cccccc
85,chr13,48689573,tttttt


In [44]:
dpileup[dpileup['pos'] == 48689492].iloc[0]['matches']

'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [24]:
dpileup2 = dpileup[['chrom', 'pos', 'matches']]

In [25]:
dpileup3 = dpileup2['matches']

In [26]:
dpileup2['base'] = dpileup3.str.len()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
dpileup2

,chrom,pos,matches,base
0,chr13,48689488,GaIaHaIaHaIaIaIaGaIaIaIaIaIaHaIaGaGaGaIaEaDaHaGa,48
1,chr13,48689489,aaaaaaaaaaaaaaaaaaaaaaaaaIaHaIaGaGaHaIaIaGaGaI...,81
2,chr13,48689490,tttttttttttttttttttttttttttttttttttttttttttttt...,85
3,chr13,48689491,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,141
4,chr13,48689492,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,105
...,...,...,...,...
82,chr13,48689570,aaaaa,5
83,chr13,48689571,cccccc,6
84,chr13,48689572,cccccc,6
85,chr13,48689573,tttttt,6


In [28]:
import math
from collections import Counter

def entropy(s):
	p, lns = Counter(s), float(len(s))
	return -sum( count/lns * math.log(count/lns, 2) for count in p.values())

In [29]:
count = ['1','2']
dent = pd.Series(count)
print(dent)
print(dent[1])

0    1
1    2
dtype: object
2


In [30]:
count = ['1','2']
ent = pd.Series(count)
print(ent)
print(ent[1])

0    1
1    2
dtype: object
2


In [31]:
i = 0
while i <= 86:
  dent[i] = entropy(dpileup3[i])
  i += 1

In [32]:
dpileup4 = dpileup2
dpileup4['Shannon_entropy'] = dent
print(dpileup4)

    chrom       pos                                            matches  base  \
0   chr13  48689488   GaIaHaIaHaIaIaIaGaIaIaIaIaIaHaIaGaGaGaIaEaDaHaGa    48   
1   chr13  48689489  aaaaaaaaaaaaaaaaaaaaaaaaaIaHaIaGaGaHaIaIaGaGaI...    81   
2   chr13  48689490  tttttttttttttttttttttttttttttttttttttttttttttt...    85   
3   chr13  48689491  aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...   141   
4   chr13  48689492  aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...   105   
..    ...       ...                                                ...   ...   
82  chr13  48689570                                              aaaaa     5   
83  chr13  48689571                                             cccccc     6   
84  chr13  48689572                                             cccccc     6   
85  chr13  48689573                                             tttttt     6   
86  chr13  48689574                                             cccccc     6   

   Shannon_entropy  
0         1.906454

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
dpileup5 = dpileup4[['chrom','pos']]
dpileup5['pos2']=dpileup4['pos']+1
dpileup5['Shannon_entropy'] = dpileup4['Shannon_entropy']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
dpileup5

,chrom,pos,pos2,Shannon_entropy
0,chr13,48689488,48689489,1.906454
1,chr13,48689489,48689490,1.4998
2,chr13,48689490,48689491,0.988811
3,chr13,48689491,48689492,1.258637
4,chr13,48689492,48689493,-0.0
...,...,...,...,...
82,chr13,48689570,48689571,-0.0
83,chr13,48689571,48689572,-0.0
84,chr13,48689572,48689573,-0.0
85,chr13,48689573,48689574,-0.0


In [35]:
dpileup5.to_csv("Mirlet7d.txt",header=False, index=False, sep='\t')

In [36]:
dpileup5sorted = dpileup5

In [37]:
dpileup5sorted.sort_values('Shannon_entropy',ascending=False)

,chrom,pos,pos2,Shannon_entropy
0,chr13,48689488,48689489,1.906454
1,chr13,48689489,48689490,1.4998
41,chr13,48689529,48689530,1.450881
39,chr13,48689527,48689528,1.343948
3,chr13,48689491,48689492,1.258637
...,...,...,...,...
38,chr13,48689526,48689527,-0.0
36,chr13,48689524,48689525,-0.0
35,chr13,48689523,48689524,-0.0
34,chr13,48689522,48689523,-0.0


In [38]:
dpileup5.to_csv("let7d.bed", sep="\t", header=None, index=False)

UCSC Genome Browser에서 Visualization 진행